In [11]:
from selenium import webdriver
import pandas as pd
import numpy as np
import os
import time
import json
import csv
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
#get list of zipcodes
zipcodes = pd.read_csv("./ZipCodes_Redfin_Scraper.csv")
list_of_zipcodes = list(zipcodes["ZipCode"].unique())

In [356]:
#to detect captcha
frame = driver.find_element_by_xpath('//iframe[contains(@src, "recaptcha")]')
driver.switch_to.frame(frame)

In [17]:
#iterate through every zipcode, for every zipcode scrape each feature and save into a csv 
for zipcode in list_of_zipcodes:
    print(zipcode)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(f"https://www.redfin.com/zipcode/{zipcode}")
    
    captcha = "Oops! It looks like our usage analysis algorithms think that you might be a robot. Accessing the redfin.com in an automated fashion is a violation of the Redfin's terms of use."
    
    #check for captcha
    if driver.find_element_by_xpath("//div[@id='txt']/p"):
        if driver.find_element_by_xpath("//div[@id='txt']/p").text == captcha      
    
    urls = []
    
    #append urls on current page
    try:
        urls+=[x.get_attribute("href") for x in driver.find_elements_by_xpath("//div[@class='PhotoSlider photoContainer']//a")]
        
    except Exception as e: 
        print(e)
        
    #loop through every page 
    try:
        total_pages = [x.get_attribute("href") for x in driver.find_elements_by_xpath("//a[@class='clickable goToPage']")]
        driver.close()
        
        if total_pages and len(total_pages)>=1:
            for page in total_pages:
                driver = webdriver.Chrome(options=chrome_options)
                driver.get(page)
                urls += [x.get_attribute("href") for x in driver.find_elements_by_xpath("//div[@class='PhotoSlider photoContainer']//a")]
                driver.close()
        
    except Exception as e: 
        print(e)
        
    print("started scraping urls")
    with open('sample1.csv', 'a+', newline='') as csvfile:
        fieldnames = ["address",
                      'State_zipcode', 
                      'price',
                      "bed",
                      "bath",
                      "square_footage",
                      "description",
                      "status",
                      "time_on_redfin",
                      "property_type",
                      "HOA_dues",
                      "year_built",
                      "community",
                      "lot_size",
                      "MLS",
                      "list_price",
                      "ppsf",
                      "redfin_est",
                      "style",
                      "lat",
                      "long",
                      "agent_name",
                      "agent_company",
                      "url",
                      "school_name",
                      "school_review",
                      "school_num_students"
                         ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()

        for url in urls:

            driver = webdriver.Chrome(options = chrome_options)
            driver.get(url)

            try:
                #Address
                address = driver.find_element_by_xpath("//div[@class='street-address']").text

                #State and zipcode
                State_zipcode = driver.find_element_by_xpath("//div[@class='dp-subtext']").text

                [price, bed, bath] = [x.text for x in driver.find_elements_by_xpath("//div[@class='statsValue']")]
                print(price, bed, bath)

                #square footage
                square_footage = driver.find_element_by_xpath("//span[@class='statsValue']").text


                #description - if continue reading click drop down 
                description = driver.find_element_by_xpath("//p[@class='text-base']//span").text

                #nullify all inital values
                status, time_on_redfin, property_type, HOA_dues, year_built, community, lot_size, MLS, list_price, ppsf, redfin_est, style = (None,)*12

                #list of home_facts  
                home_facts = driver.find_elements_by_xpath("//div[@class='keyDetail font-weight-roman font-size-base']//span")

                for i in range(len(home_facts)):
                    text = home_facts[i].text

                    if text == "Status":
                        status = home_facts[i+1].text

                    elif text == "Time on Redfin": 
                        time_on_redfin = home_facts[i+1].text

                    elif text == "Property Type":
                        property_type = home_facts[i+1].text

                    elif text == "HOA Dues":
                        HOA_dues = home_facts[i+1].text

                    elif text == "Year Built":
                        year_built = home_facts[i+1].text

                    elif text == "Community":
                        community = home_facts[i+1].text

                    elif text == "Lot Size":
                        lot_size = home_facts[i+1].text

                    elif text == "MLS#":
                        MLS = home_facts[i+1].text

                    elif text == "List Price":
                        list_price = home_facts[i+1].text

                    elif text == "Price/Sq.Ft.":
                        ppsf = home_facts[i+1].text

                    elif text == "Redfin Estimate":
                        redfin_est = home_facts[i+1].text

                    elif text == "Style":
                        style = home_facts[i+1].text
                        

                #latitude 
                lat = json.loads(driver.find_element_by_xpath("//div[@class='Section AddressBannerSectionV2 white-bg not-omdp']//script").get_attribute('innerHTML'))["geo"]["latitude"]

                #long 
                long = json.loads(driver.find_element_by_xpath("//div[@class='Section AddressBannerSectionV2 white-bg not-omdp']//script").get_attribute('innerHTML'))["geo"]["longitude"]

                #agent name
                agent_name = driver.find_element_by_xpath("//span[@class='agent-name']").text

                #angent company 
                agent_company = driver.find_element_by_xpath("//div[@class='aaq-question-stage-agent-info']//div").text

                #school name
                school_name = driver.find_element_by_xpath("//tr[@class='schools-table-row']//div[@class='school-title']").get_attribute('innerHTML')
                
                #school review 
                school_review = driver.find_element_by_xpath("//tr[@class='schools-table-row']//span[@class='rating-num']").get_attribute('innerHTML')
                
                #num of students
                school_num_students = driver.find_element_by_xpath("//td[@class='num-students-col']//div[@class='students']").get_attribute('innerHTML')
                
                
                list_table = { 
                    "address": address, 
                    "State_zipcode":State_zipcode,
                    "price":price,
                    "bed":bed, 
                    "bath":bath,
                    "square_footage":square_footage,
                    "description":description,
                    "status":status,
                    "time_on_redfin":time_on_redfin,
                    "property_type":property_type,
                    "HOA_dues":HOA_dues,
                    "year_built":year_built,
                    "community":community,
                    "lot_size":lot_size,
                    "MLS":MLS,
                    "list_price":list_price,
                    "ppsf":ppsf,
                    "redfin_est":redfin_est,
                    "style":style,
                    "lat":lat,
                    "long":long,
                    "agent_name":agent_name,
                    "agent_company":agent_company,
                    "url":url,
                    "school_name": school_name, 
                    "school_review": school_review,
                    "school_num_students": school_num_students
                }

                writer.writerow(list_table)

            except Exception as e: 
                print(e, url)

            driver.close()
            time.sleep(1)
            
        
      


20008
started scraping urls


TypeError: expected str, bytes or os.PathLike object, not Options

In [7]:
x = [1,2,3,4,5,6,7,8,9,10]
import numpy as np

result = np.array_split(x, 4)
result

[array([1, 2, 3]), array([4, 5, 6]), array([7, 8]), array([ 9, 10])]